In [1]:
import sys
import os
import cv2
import numpy as np
import time
from openvino.inference_engine import IENetwork, IECore
from matplotlib import pyplot as plt
%matplotlib inline

In [49]:
workspace_dir = '/home/manuel/Bachelor_Arbeit/workspace'
dataset = 'OI_Animals_Augmented_9_2000/'
model = 'faster_rcnn_inception_v2_coco_2018_01_28_out/'
model_xml = os.path.join(workspace_dir, dataset, model, 'open_vino', 'frozen_inference_graph.xml')
model_bin = os.path.join(workspace_dir, dataset, model, 'open_vino', 'frozen_inference_graph.bin')
labels = [l.strip() for l in open(os.path.join(workspace_dir, dataset, 'classes.txt')).readlines()]
test_image_dir = '/home/manuel/Bachelor_Arbeit/test'
#test_images_dir = os.path.join(workspace_dir, dataset, 'validation')
images = [
    os.path.join(test_image_dir, img) for img in os.listdir(test_image_dir) if img[-3:] == 'jpg']

result_dir = os.path.join(test_image_dir, 'results')

assert os.path.isdir(result_dir)
assert os.path.isfile(model_bin)
assert os.path.isfile(model_xml)
assert os.path.isdir(test_image_dir)
print(len(images))
labels

16


['Brown_bear',
 'Deer',
 'Fox',
 'Goat',
 'Hedgehog',
 'Owl',
 'Rabbit',
 'Raccoon',
 'Squirrel']

In [7]:
ie = IECore()
net = IENetwork(model=model_xml, weights=model_bin)
exec_net = ie.load_network(network=net, num_requests=2, device_name='MYRIAD')

In [50]:
img_info_input_blob = None
feed_dict = {}
for blob_name in net.inputs:
    if len(net.inputs[blob_name].shape) == 4:
        input_blob = blob_name
    elif len(net.inputs[blob_name].shape) == 2:
        img_info_input_blob = blob_name
    else:
        raise RuntimeError("Unsupported {}D input layer '{}'. Only 2D and 4D input layers are supported"
                               .format(len(net.inputs[blob_name].shape), blob_name))

assert len(net.outputs) == 1, "Demo supports only single output topologies"

out_blob = next(iter(net.outputs))
n, c, h, w = net.inputs[input_blob].shape
if img_info_input_blob:
    feed_dict[img_info_input_blob] = [h, w, 1]

cur_request_id = 0
next_request_id = 1

frame = cv2.imread(images[0])
initial_h, initial_w = frame.shape[: 2]
img_name = images[0].split('/')[-1]


In [53]:
i = 0
n_images = len(images)
images_to_infer = []
max_images_to_infer = 45
remove_ind = 1
liste_str = ''
last_frame_processed = False

while True:
#for img_path in images:
    # in_frame = None
    
    time.sleep(0.08) # hier bewegungserkennung
    
    if i < n_images:
        images_to_infer.insert(0, cv2.imread(images[i]))
        liste_str += '='
        #print(str(len(images_to_infer)) + ': ' + liste_str)
        if len(images_to_infer) > max_images_to_infer:
            del images_to_infer[-remove_ind]
            liste_str = liste_str[:-1]
            #rint(str(len(images_to_infer)) + ': ' + liste_str)
            remove_ind += 1
        else:
            #print('old ', str(remove_ind))
            remove_ind = 1
            #print('new ', str(remove_ind))
        # print('adding image to list. List size: ' + str(len(images_to_infer)))
        # print('image ===> INFERLIST')
        
        
        
    elif not images_to_infer:
        print('liste leer')
        # break
        
    print('status ' + str(exec_net.requests[cur_request_id].wait(0)))
    
    if exec_net.requests[cur_request_id].wait(0) == 0 or exec_net.requests[cur_request_id].wait(0) == -11:
        # wait(0) ohne timeout, also sofrtiges return
        # OK = 0
        # INFER_NOT_STARTED = -11
        # RESULT_NOT_READY = -9
        
        if images_to_infer: # lsit nicht leer
        
            next_frame = images_to_infer.pop()
            liste_str = liste_str[:-1]
            #print(str(len(images_to_infer)) + ': ' + liste_str)
            #if remove_ind > 1:
            #    remove_ind -= 1

            #print('taking image from list. List size: ' + str(len(images_to_infer)))
            # print('           INFERLIST ===> process')

            next_initial_h, next_initial_w = next_frame.shape[: 2]

            in_frame = cv2.resize(next_frame, (w, h))
            in_frame = in_frame.transpose((2, 0, 1))
            in_frame = in_frame.reshape((n, c, h, w))
            feed_dict[input_blob] = in_frame
            exec_net.start_async(request_id=next_request_id, inputs=feed_dict)
            last_frame_processed = False
        
        if exec_net.requests[cur_request_id].wait(0) != -11 and not last_frame_processed:
            
            res = exec_net.requests[cur_request_id].outputs[out_blob]
            for obj in res[0][0]:
                if obj[2] > 0.5:
                    xmin = int(obj[3] * initial_w)
                    ymin = int(obj[4] * initial_h)
                    xmax = int(obj[5] * initial_w)
                    ymax = int(obj[6] * initial_h)
                    class_id = int(obj[1])
                    color = (min(class_id * 12.5, 255),
                                 min(class_id * 7, 255), min(class_id * 5, 255))

                    cv2.rectangle(frame, (xmin, ymin),
                                      (xmax, ymax), (255, 255, 0), 2)
                    det_label = labels[class_id - 1] if labels else str(
                            class_id)
                    cv2.putText(frame, det_label + ' ' + str(round(obj[2] * 100, 1)) + ' %', (xmin, ymin - 7),
                                    cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 0), 1)
            #print('save image to ', os.path.join(test_image_dir, 'results', img_name))
            cv2.imwrite(os.path.join(test_image_dir, 'results', 'img_' + str(i) + '.jpg'), frame)
                
            cur_request_id, next_request_id = next_request_id, cur_request_id
            frame = next_frame
            initial_w = next_initial_w
            initial_h = next_initial_h
            
            if not images_to_infer:
                last_frame_processed = True
                
                
        #img_name = next_img_name

    #elif in_frame is not None:
        #images_to_infer.insert(0, in_frame)
        #print('adding image to queue with lenght ' + str(len(images_to_infer)))
    #cv2.imshow("Detection Results", frame)

    
    i += 1
    
#     key = cv2.waitKey(0)
#     if key == 27:
#         break

#cv2.destroyAllWindows()

status 0
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status 0
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status 0
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status 0
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status 0
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
status -9
statu

KeyboardInterrupt: 